**MOUNTING OF THE GOOGLE DRIVE TO IMPORT THE DATASETS**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Performing NLP on Legal tech is very critical because of non availiablity of the datasets. And hence processing of dataset also play a key role in such domains. Hence I have use 3 datasets of different formats so that I can engage myself to get in hand experiece before starting the actual thesis**

**MANDATORY CELLS TO BE RUN - IRRESPECTIVE OF THE DATASET**

In [0]:
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import warnings
import gensim 
import glob
from gensim.models import Word2Vec
nltk.download('stopwords')
import re
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import PunktSentenceTokenizer
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('wordnet')
warnings.filterwarnings(action = 'ignore')
from string import punctuation

#--------------------------------------------------------------------------------------#

#Python program to convert a list to string 
    
# Function to convert   
def listToString(s):  
    
    # initialize an empty string 
    text = ""  
    
    # traverse in the string   
    for ele in s:  
        text += ele   
    
    # return string   
    return text  


#---------------------------------------------------------------------------------------#

# For conversion from string to list because while chunking we will convert list to string
def Convert(string): 
    li = list(string.split(" ")) 
    return li

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


**DATASET - 1 (IMPORTING AND OTHER VARIABLE CONVERSIONS)**

I have used beautiful soup library for extracting and parsing the content from xml files. The dataset had 3 directories **fulltext** containing the whole court proceedings, **summary** containing only technical abstract of the each case filed and the **class** containing the history and details of the cases.

I have used the content under the tag **citances** which is from the file summary because it had the most informative part of the cases such as the **participanting parties**,**location** where the case proceedings were held and other details such as time. 

Dataset can be downloaded from - https://archive.ics.uci.edu/ml/datasets/Legal+Case+Reports

Note - The data set has 4000 recorded cases but for the initial task I am using randomly chosen 45 cases from it. The reason is that using a sample of data from it will not affect the accuracy of basic nlp tasks.

In [0]:

con = " "
for file in glob.glob("/content/drive/My Drive/Test_corpus_SUMM/*"):
        with open(file, 'r') as infile2:
                con += listToString(infile2.readlines()) 
                
soup = BeautifulSoup(con, 'lxml')
contents = " "
required0 = soup.find_all("citances")
for i in required0:
    contents+=i.get_text()
print(contents)

    # The chunking function is slowing down the execution if I use the whole dataset therefore I am using a single arbitrary case
    # from the list.
with open('/content/drive/My Drive/Test_corpus_SUMM/06_31.xml', "r") as f:
        contents_read = f.read()
        soup = BeautifulSoup(contents_read, 'lxml') 
required1 = soup.find_all("citances")
train_sentence_sample = []
for i in required1:
    train_sentence_sample.append(i.get_text())
    
  # The chunking function is slowing down the execution if I use the whole dataset therefore I am using a single arbitrary case
# from the list.  
with open('/content/drive/My Drive/Test_corpus_SUMM/06_36.xml', "r") as f:
        contents_read = f.read()
        soup = BeautifulSoup(contents_read, 'lxml')
required2 = soup.find_all("citances")
test_sentence_sample = []
for i in required2:
    test_sentence_sample.append(i.get_text())

 


 
 38 Mitre 10 claims that these circumstances are very clearly analogous to those in which an interlocutory injunction was granted in favour of Clark Rubber by Goldberg J in Intellectual Property Pty Ltd v Mygroups Pty Ltd. [9] In that case, Goldberg J restrained an operator of a foam-rubber product store in Victoria from trading whilst the store's exterior was painted in bright yellow, with red and blue lettering for signage. That was the same combination of colours used by Clark Rubber on its stores selling the same or similar products. The evidence was that Clark Rubber had 70 stores Australia-wide, 65 of which used that colour scheme. It had traded in Australia for 60 years. The selection of colours was accepted by his Honour to be, rather than functional or utilitarian, idiosyncratic. His Honour accepted that on the evidence in that case there was a serious question to be tried that: 
 
 
 ... the Clark Rubber colour scheme is quite distinctive and that it creates a strong im

**DATASET - 2 (IMPORTING AND OTHER TYPE CONVERSIONS)**

I wanted to try out different kind of datasets, hence this dataset is regarding credit card agreements which are included as a part of legal documents. Here again I have limited my dataset to Citi bank credit card agreements, though the dataset has records of different banks.

This dataset had the legal documents in the pdf format but the libraries cannot scrap the content from that format so I used PyPDF library to extract the content.

Dataset link - https://files.consumerfinance.gov/a/assets/CCADB_Snapshot_2015_Q4.zip

In [0]:
pip install PyPDF2

     |████████████████████████████████| 81kB 2.7MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-cp36-none-any.whl size=61085 sha256=188a9df027f255553440c97b2638d10d8e47b4f4ebd66dc64a7ebccf9c31521f
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2


In [0]:
import PyPDF2
string = " "
pdfFileObj = open('/content/drive/My Drive/Citi/Citi AAdvantage Platinum Select Visa.pdf','rb')    
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

for i in range(0, pdfReader.numPages ):
    pageObj = pdfReader.getPage(i)
    string+=pageObj.extractText()
    

#print(type(string))
contents = string
#contents = Convert(string)
print(type(contents))


train_sentence_sample = string

train_sentence_sample = Convert(train_sentence_sample)


# This is second file for chunking purpose
string = " "
pdfFileObj = open('/content/drive/My Drive/Citi/Citi AAdvantage Platinum Select Card.pdf','rb')     #'rb' for read binary mode
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

for i in range(0, pdfReader.numPages ):
    pageObj = pdfReader.getPage(i)
    string+=pageObj.extractText()
    
print(type(string))
test_sentence_sample = string

<class 'str'>
<class 'str'>


**DATASET - 3 (IMPORTING AND OTHER TYPE CONVERSIONS)**

I read through few papers where NLP is used for Legal Tech and I found many of them scraping the content from the website as it is hard to find relavant dataset of legal documents. So my idea was to implement the same for a good try. I was unsure of which website is legally secure to scrap hence I chose the wikipedia page of **Indian Penal Code** which details about every section of the law available under the jurisdiction.  

In [0]:
import requests
website_url = requests.get("https://en.wikipedia.org/wiki/Indian_Penal_Code").text

from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())

context = []
content1 = soup.find_all('p')
for row in content1:          
    context.append(row.get_text())
    
content2 = soup.find_all('td')
for row in content2:          
    context.append(row.get_text())
    
content3 = soup.find_all('li')
for row in content3:         
    context.append(row.get_text())


train_sentence_sample = context[0:500]
test_sentence_sample = context[501:800]

context = listToString(context)
#context = context.replace('\n','')
contents = context

print(type(contents))

<class 'str'>


**TOKENIZATION**

In [0]:
contents = contents.split()
print(contents)
print(len(contents))

['The', 'Indian', 'Penal', 'Code', '(IPC)', 'is', 'the', 'official', 'criminal', 'code', 'of', 'India.', 'It', 'is', 'a', 'comprehensive', 'code', 'intended', 'to', 'cover', 'all', 'substantive', 'aspects', 'of', 'criminal', 'law.', 'The', 'code', 'was', 'drafted', 'in', '1860', 'on', 'the', 'recommendations', 'of', 'first', 'law', 'commission', 'of', 'India', 'established', 'in', '1834', 'under', 'the', 'Charter', 'Act', 'of', '1833', 'under', 'the', 'Chairmanship', 'of', 'Lord', 'Thomas', 'Babington', 'Macaulay.[1][2][3]', 'It', 'came', 'into', 'force', 'in', 'British', 'India', 'during', 'the', 'early', 'British', 'Raj', 'period', 'in', '1862.', 'However,', 'it', 'did', 'not', 'apply', 'automatically', 'in', 'the', 'Princely', 'states,', 'which', 'had', 'their', 'own', 'courts', 'and', 'legal', 'systems', 'until', 'the', '1940s.', 'The', 'Code', 'has', 'since', 'been', 'amended', 'several', 'times', 'and', 'is', 'now', 'supplemented', 'by', 'other', 'criminal', 'provisions.', 'After

In [0]:
#for i in contents:
    #contents = nltk.word_tokenize(i)
    #print(contents)

**STOP WORDS REMOVAL**

In [0]:
cachedStopWords = stopwords.words("english")
contents = [word for word in contents if word not in cachedStopWords]
print(contents)
print(len(contents))

['The', 'Indian', 'Penal', 'Code', '(IPC)', 'official', 'criminal', 'code', 'India.', 'It', 'comprehensive', 'code', 'intended', 'cover', 'substantive', 'aspects', 'criminal', 'law.', 'The', 'code', 'drafted', '1860', 'recommendations', 'first', 'law', 'commission', 'India', 'established', '1834', 'Charter', 'Act', '1833', 'Chairmanship', 'Lord', 'Thomas', 'Babington', 'Macaulay.[1][2][3]', 'It', 'came', 'force', 'British', 'India', 'early', 'British', 'Raj', 'period', '1862.', 'However,', 'apply', 'automatically', 'Princely', 'states,', 'courts', 'legal', 'systems', '1940s.', 'The', 'Code', 'since', 'amended', 'several', 'times', 'supplemented', 'criminal', 'provisions.', 'After', 'partition', 'British', 'Indian', 'Empire,', 'Indian', 'Penal', 'Code', 'inherited', 'successor', 'states,', 'Dominion', 'India', 'Dominion', 'Pakistan,', 'continues', 'independently', 'Pakistan', 'Penal', 'Code.', 'The', 'Ranbir', 'Penal', 'Code', '(R.P.C)', 'applicable', 'Jammu', 'Kashmir', 'also', 'based'

**EXTRACTING ONLY ALPHANUMERIC CONTENT**

In [0]:
p = re.compile('[a-zA-Z0-9]+')
filtered_tokens_1 = list(filter(lambda token: p.match(token), contents))
#filtered_tokens
print(len(filtered_tokens_1))
print(filtered_tokens_1)

4317
['The', 'Indian', 'Penal', 'Code', 'official', 'criminal', 'code', 'India.', 'It', 'comprehensive', 'code', 'intended', 'cover', 'substantive', 'aspects', 'criminal', 'law.', 'The', 'code', 'drafted', '1860', 'recommendations', 'first', 'law', 'commission', 'India', 'established', '1834', 'Charter', 'Act', '1833', 'Chairmanship', 'Lord', 'Thomas', 'Babington', 'Macaulay.[1][2][3]', 'It', 'came', 'force', 'British', 'India', 'early', 'British', 'Raj', 'period', '1862.', 'However,', 'apply', 'automatically', 'Princely', 'states,', 'courts', 'legal', 'systems', '1940s.', 'The', 'Code', 'since', 'amended', 'several', 'times', 'supplemented', 'criminal', 'provisions.', 'After', 'partition', 'British', 'Indian', 'Empire,', 'Indian', 'Penal', 'Code', 'inherited', 'successor', 'states,', 'Dominion', 'India', 'Dominion', 'Pakistan,', 'continues', 'independently', 'Pakistan', 'Penal', 'Code.', 'The', 'Ranbir', 'Penal', 'Code', 'applicable', 'Jammu', 'Kashmir', 'also', 'based', 'Code,', 'How

**STEMMING - PORTER STEMMER ALGORITHM**

In [0]:
filtered_tokens_2 = list(map(lambda token: PorterStemmer().stem(token), filtered_tokens_1))
print(filtered_tokens_2)
    

['the', 'indian', 'penal', 'code', 'offici', 'crimin', 'code', 'india.', 'It', 'comprehens', 'code', 'intend', 'cover', 'substant', 'aspect', 'crimin', 'law.', 'the', 'code', 'draft', '1860', 'recommend', 'first', 'law', 'commiss', 'india', 'establish', '1834', 'charter', 'act', '1833', 'chairmanship', 'lord', 'thoma', 'babington', 'macaulay.[1][2][3]', 'It', 'came', 'forc', 'british', 'india', 'earli', 'british', 'raj', 'period', '1862.', 'however,', 'appli', 'automat', 'princ', 'states,', 'court', 'legal', 'system', '1940s.', 'the', 'code', 'sinc', 'amend', 'sever', 'time', 'supplement', 'crimin', 'provisions.', 'after', 'partit', 'british', 'indian', 'empire,', 'indian', 'penal', 'code', 'inherit', 'successor', 'states,', 'dominion', 'india', 'dominion', 'pakistan,', 'continu', 'independ', 'pakistan', 'penal', 'code.', 'the', 'ranbir', 'penal', 'code', 'applic', 'jammu', 'kashmir', 'also', 'base', 'code,', 'however,', 'success', 'passag', 'jammu', 'kashmir', 'reorganis', 'bill,', '2

**LEMMATIZATION**

In [0]:
lemmatizer = WordNetLemmatizer()

filtered_tokens_3 = list(map(lambda token: lemmatizer.lemmatize(token), filtered_tokens_2))
print(filtered_tokens_3)

['the', 'indian', 'penal', 'code', 'offici', 'crimin', 'code', 'india.', 'It', 'comprehens', 'code', 'intend', 'cover', 'substant', 'aspect', 'crimin', 'law.', 'the', 'code', 'draft', '1860', 'recommend', 'first', 'law', 'commiss', 'india', 'establish', '1834', 'charter', 'act', '1833', 'chairmanship', 'lord', 'thoma', 'babington', 'macaulay.[1][2][3]', 'It', 'came', 'forc', 'british', 'india', 'earli', 'british', 'raj', 'period', '1862.', 'however,', 'appli', 'automat', 'princ', 'states,', 'court', 'legal', 'system', '1940s.', 'the', 'code', 'sinc', 'amend', 'sever', 'time', 'supplement', 'crimin', 'provisions.', 'after', 'partit', 'british', 'indian', 'empire,', 'indian', 'penal', 'code', 'inherit', 'successor', 'states,', 'dominion', 'india', 'dominion', 'pakistan,', 'continu', 'independ', 'pakistan', 'penal', 'code.', 'the', 'ranbir', 'penal', 'code', 'applic', 'jammu', 'kashmir', 'also', 'base', 'code,', 'however,', 'success', 'passag', 'jammu', 'kashmir', 'reorganis', 'bill,', '2

**POS TAGGING**

In [0]:
pos_tagged = nltk.pos_tag(filtered_tokens_3)
print(pos_tagged)
    

[('the', 'DT'), ('indian', 'JJ'), ('penal', 'JJ'), ('code', 'NN'), ('offici', 'IN'), ('crimin', 'NN'), ('code', 'NN'), ('india.', 'IN'), ('It', 'PRP'), ('comprehens', 'VBZ'), ('code', 'JJ'), ('intend', 'VBP'), ('cover', 'NN'), ('substant', 'NN'), ('aspect', 'JJ'), ('crimin', 'NN'), ('law.', 'VBD'), ('the', 'DT'), ('code', 'NN'), ('draft', 'NN'), ('1860', 'CD'), ('recommend', 'NN'), ('first', 'JJ'), ('law', 'NN'), ('commiss', 'NN'), ('india', 'VBP'), ('establish', 'VB'), ('1834', 'CD'), ('charter', 'NN'), ('act', 'NN'), ('1833', 'CD'), ('chairmanship', 'NN'), ('lord', 'NN'), ('thoma', 'IN'), ('babington', 'NN'), ('macaulay.[1][2][3]', 'NN'), ('It', 'PRP'), ('came', 'VBD'), ('forc', 'JJ'), ('british', 'JJ'), ('india', 'NN'), ('earli', 'NN'), ('british', 'JJ'), ('raj', 'NN'), ('period', 'NN'), ('1862.', 'CD'), ('however,', 'NN'), ('appli', 'NN'), ('automat', 'NN'), ('princ', 'NN'), ('states,', 'JJ'), ('court', 'NN'), ('legal', 'JJ'), ('system', 'NN'), ('1940s.', 'CD'), ('the', 'DT'), ('co

In [0]:
#directory = '/content/drive/My Drive/Test_corpus'
#for filename in os.listdir(directory):
   # if '.ipynb_checkpoints' in filename:
        #print("yes")
    #else:
        #print("no")
    

**CHUNKING**

In [0]:
train_sentence_sample = listToString(train_sentence_sample)
test_sentence_sample = listToString(test_sentence_sample)

In [0]:
custom_sent_tokenizer = PunktSentenceTokenizer(train_sentence_sample)
tokenized = custom_sent_tokenizer.tokenize(test_sentence_sample)

In [0]:
print(type(tokenized))

<class 'list'>


In [0]:
def chunking_function():
    try:
        for i in tokenized:
            words = nltk.word_tokenize(i)
            tagged = nltk.pos_tag(words)
            chunkGram = r"""Chunk: {<RB.?>*<VB.?>*<NNP>+<NN>?}"""
            chunkParser = nltk.RegexpParser(chunkGram)
            chunked = chunkParser.parse(tagged)
            
            print(chunked)
            for subtree in chunked.subtrees(filter=lambda t: t.label() == 'Chunk'):
                print(subtree)

            chunked.draw()

    except Exception as e:
        print(str(e))

In [0]:
chunking_function()

(S
  ^/NN
  ``/``
  (Chunk Supreme/NNP Court/NNP)
  's/POS
  decision/NN
  on/IN
  Section/NN
  377/CD
  :/:
  Separate/JJ
  decision/NN
  of/IN
  5/CD
  Judges/NNS
  (Chunk [/VBP Read/NNP Judgement/NNP ]/NNP)
  ''/''
  ./.)
(Chunk Supreme/NNP Court/NNP)
(Chunk [/VBP Read/NNP Judgement/NNP ]/NNP)
no display name and no $DISPLAY environment variable


**SAMPLE STRING TO CHECK THE NER FUNCTIONING**

In [0]:
sample_array = ['''However, these documents suggest, with respect, that: (i) Neither counsel for the taxpayer, nor counsel for the Commissioner, nor the Tribunal was aware that the taxpayer had not included any ground in his notice of objection going to the additional tax imposed at label D of the amended notice of assessment'''
                          '''In Commonwealth Bank Officers Superannuation Corporation Pty Ltd v Commissioner of Taxation [2005] FCAFC 244 (30 November 2005, unreported) the Full Court of this Court observed at [29] --- [30]:
                           '...Section 43 empowers the Tribunal to exercise all the powers and discretions conferred upon the original decision-maker, provided it does so for the purpose of reviewing a decision'''       
                         ]


test_array = ['''I mean, it couldn't be the case that having regard to the number of taxpayers and the way the Commissioner must administer the Act that not to put at least copies of the agreements, that that was full disclosure'''
                      '''Where a taxpayer omits an item of assessable income from their return, s.223(7) deems the taxpayer to have made a statement in their return to the effect that they did not derive that item of income'''
                      '''In his written submissions in reply, the Commissioner refocussed his argument, to where it always should have been focussed, on subs 43(1) of the AAT Act'''
                      '''In Commonwealth Bank Officers Superannuation Corporation Pty Ltd v Commissioner of Taxation [2005] FCAFC 244 (30 November 2005, unreported) the Full Court of this Court observed at [29] --- [30]:
                           '...Section 43 empowers the Tribunal to exercise all the powers and discretions conferred upon the original decision-maker, provided it does so for the purpose of reviewing a decision'''       
                     ]

dummy_my_example = ['''Only France and Italy backed Fischler's proposal''']

**NAMED ENITITY RECOGNITITON**

In [0]:
def NER():
    try:
        for item in filtered_tokens_1:
            tokenized = nltk.word_tokenize(item)
            tagged = nltk.pos_tag(tokenized)
            #print(tagged)

            namedEnt = nltk.ne_chunk(tagged)
            print(namedEnt)

    except Exception as e:
        print(str(e))

In [0]:
NER()

(S The/DT)
(S (GPE Indian/JJ))
(S Penal/JJ)
(S (GPE Code/NN))
(S official/NN)
(S criminal/JJ)
(S code/NN)
(S (GPE India/NNP) ./.)
(S It/PRP)
(S comprehensive/NN)
(S code/NN)
(S intended/VBN)
(S cover/NN)
(S substantive/JJ)
(S aspects/NNS)
(S criminal/JJ)
(S law/NN ./.)
(S The/DT)
(S code/NN)
(S drafted/VBN)
(S 1860/CD)
(S recommendations/NNS)
(S first/RB)
(S law/NN)
(S commission/NN)
(S (GPE India/NNP))
(S established/VBN)
(S 1834/CD)
(S Charter/NN)
(S Act/NNP)
(S 1833/CD)
(S Chairmanship/NN)
(S (GPE Lord/NN))
(S (GPE Thomas/NNP))
(S (PERSON Babington/NNP))
(S
  (GPE Macaulay/NNP)
  ./.
  [/CC
  1/CD
  ]/JJ
  [/$
  2/CD
  ]/NNP
  [/VBD
  3/CD
  ]/NN)
(S It/PRP)
(S came/VBD)
(S force/NN)
(S (GPE British/JJ))
(S (GPE India/NNP))
(S early/RB)
(S (GPE British/JJ))
(S (GPE Raj/NN))
(S period/NN)
(S 1862/CD ./.)
(S However/RB ,/,)
(S apply/VB)
(S automatically/RB)
(S Princely/RB)
(S states/NNS ,/,)
(S courts/NNS)
(S legal/JJ)
(S systems/NNS)
(S 1940s/CD ./.)
(S The/DT)
(S (GPE Code/NN))
(S s

**EXTRACTING CONTENTS FROM THE DATASET - 1 (ONLY THE LEGISLATION TITLES ARE EXTRACTED)**

The purpose of extracting only the ACT's is to determine which has the maximum frequency in the cases registered. From the lists we can arrive at a conclusion that Migration act has been registered more frequently. 

In [0]:
pon = " "
xtends = []
#required3 = []
for file in glob.glob("/content/drive/My Drive/Test_corpus_SUMM/*"):
        with open(file, 'r') as infile:
                pon += listToString(infile.readlines())

soup = BeautifulSoup(pon, 'lxml')
required3 = soup.find_all("title", attrs = { "id" : "l0"})
for i in required3:
    xtends.append(i.get_text())
xtends

['CORPORATIONS ACT 2001 ',
 'TRADE PRACTICES ACT 1974 ',
 'INDUSTRIAL RELATIONS ACT 1988 ',
 'TRADE MARKS ACT 1995 ',
 'MUTUAL ASSISTANCE IN CRIMINAL MATTERS LEGISLATION AMENDMENT BILL 1996 ',
 'TRADE PRACTICES ACT 1974 ',
 'ACTS INTERPRETATION ACT 1901 ',
 'TRADE PRACTICES ACT 1974 ',
 'MIGRATION ACT 1958 ',
 'CORPORATIONS ACT 2001 ',
 'BANKRUPTCY ACT 1966 ',
 'FEDERAL COURT RULES 1979 ',
 'MIGRATION ACT 1958 ',
 'PATENTS ACT 1990 ',
 'TRADE PRACTICES ACT 1974 ',
 'MIGRATION ACT 1958 ',
 'NATIVE TITLE ACT 1993 ',
 'BANKRUPTCY ACT 1966 ',
 'ADMINISTRATIVE APPEALS TRIBUNAL ACT 1975 ',
 'MIGRATION ACT 1958 ',
 'TRADE PRACTICES ACT 1974 ',
 'ADMINISTRATIVE APPEALS TRIBUNAL ACT 1975 ',
 'MIGRATION ACT 1958 ',
 'MIGRATION ACT 1958 ',
 'BANKRUPTCY ACT 1966 ',
 'FEDERAL COURT RULES 1979 ',
 'TRADE PRACTICES ACT 1974 ',
 'JUDICIARY ACT 1903 ',
 'FEDERAL COURT RULES 1979 ',
 'FEDERAL COURT RULES 1979 ',
 'ADMINISTRATIVE APPEALS TRIBUNAL ACT 1975 ',
 'FEDERAL COURT RULES 1979 ',
 'FEDERAL COURT 

In [0]:
def listToStr(s):
    string = ' '.join(map(str, s ))
    return string

**FINDING KEYWORDS FUNCTION**

The purpose of the function is to extract the keywords such as the noun and verbs from the content. These words will be further used for determining if they occur in other cases registered under Migration Act using Tf vectorizer.

In [0]:
def find_keywords(path):
    with open(path, "r") as f:
            contents_read = f.read()
            soup = BeautifulSoup(contents_read, 'lxml') 
    required1 = soup.find_all("citances")
    train_sentence_sample = []

    for i in required1:
        train_sentence_sample.append(i.get_text())

    nouns = []
    train_sentence_sample=listToString(train_sentence_sample)
    sentences = nltk.sent_tokenize(train_sentence_sample) 

    for sentence in sentences:
        for word,pos in nltk.pos_tag(nltk.word_tokenize(str(sentence))):
            if (pos == 'NN' or pos == 'VB' or pos == 'VBN'):
                nouns.append(word)

    filtered_nouns = list(filter(lambda a: a != ']' , nouns))
    filtered_nouns = list(filter(lambda a: a != '[' , nouns))

    filtered_keywords = []
    filtered_keywords_migration = []
    for word in filtered_nouns:  
        if word not in filtered_keywords:                
            filtered_keywords.append(word)

    for i in range(0, len(filtered_keywords)):
        if len(filtered_keywords[i]) > 3:
            filtered_keywords_migration.append(filtered_keywords[i])
    
    return filtered_keywords_migration

These are the documents which are related to Migration act 

In [0]:
from_06_12 = listToStr(find_keywords('/content/drive/My Drive/Test_corpus_SUMM/06_12.xml'))
from_06_7 = listToStr(find_keywords('/content/drive/My Drive/Test_corpus_SUMM/06_7.xml'))
from_06_3 = listToStr(find_keywords('/content/drive/My Drive/Test_corpus_SUMM/06_3.xml'))
from_06_16 = listToStr(find_keywords('/content/drive/My Drive/Test_corpus_SUMM/06_16.xml'))

#migration_documents = [from_06_12, from_06_7, from_06_3, from_06_16]

#type(migration_documents)

**COUNT VECTORIZING FUNCTION**

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

count_vectorizer = CountVectorizer(stop_words='english')
count_vectorizer = CountVectorizer()
sparse_matrix = count_vectorizer.fit_transform([from_06_12, from_06_7, from_06_3, from_06_16])

doc_term_matrix = sparse_matrix.todense()
df = pd.DataFrame(doc_term_matrix, 
                  columns=count_vectorizer.get_feature_names(), 
                  index=['from_06_12', 'from_06_7', 'from_06_3', 'from_06_16'])
df

,170,841of,abandoned,ability,absence,accept,accepted,accident,accommodation,accord,account,acknowledge,acknowledged,acted,action,actor,addition,address,addressed,adequacy,advanced,affirmed,agree,allegation,allow,analysis,anyone,appeal,appealed,appellant,applicant,application,applied,area,argue,argument,arise,articulated,ascertain,aspect,...,subrule,success,suggest,suggested,suggestion,support,supporter,suppression,supra,surmise,sustained,system,take,termed,terminated,termination,test,thought,tort,traffic,treated,trial,tribunal,turn,understanding,union,unwillingness,used,view,violence,virtue,visa,want,weight,well,wife,witness,woman,work,written
from_06_12,1,0,1,0,0,0,1,1,0,1,0,0,0,1,2,0,1,0,0,0,0,0,0,1,1,0,1,0,0,0,1,1,1,1,1,1,0,0,1,1,...,1,0,0,0,1,1,0,1,1,1,1,0,1,1,1,0,1,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
from_06_7,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
from_06_3,0,0,0,1,1,1,0,0,1,0,1,1,1,0,0,1,0,1,1,1,1,1,1,0,1,1,0,1,0,1,1,1,0,0,0,1,1,1,0,1,...,0,1,1,1,0,1,1,0,1,0,0,1,1,0,0,0,1,1,0,0,1,0,1,0,0,0,1,1,1,1,1,0,0,1,1,1,0,1,1,1
from_06_16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


**WORD EMBEDDINGS (WORD2VEC AND SKIPGRAM)**

In [0]:
 GENSIM_MODEL = gensim.models.Word2Vec([filtered_tokens_1], min_count = 1,  
                              size = 100, window = 5)

In [0]:
SKIP_GRAM = gensim.models.Word2Vec([filtered_tokens_1], min_count = 1, size = 100, 
                                             window = 5, sg = 1)

In [0]:
result_GENSIM_MODEL = GENSIM_MODEL.similar_by_word("Indian")
print("{}: {:.4f} {}: {:.4f}" .format(*result_GENSIM_MODEL[0], *result_GENSIM_MODEL[1]))

Act,: 0.3497 Act: 0.3191


In [0]:
result_SKIP_GRAM = SKIP_GRAM.similar_by_word("account")
print("{}: {:.4f} {}: {:.4f}" .format(*result_SKIP_GRAM[0], *result_SKIP_GRAM[1]))

may: 0.9741 must: 0.9740


**END OF THE PROJECT BELOW THIS IS**
**ROUGH WORK DO NOT EVALUATE**

In [0]:
pip install PyPDF2

In [0]:
import PyPDF2

def getPDFContent(path):
    content = " "
    # Load PDF into pyPDF
    pdf = PyPDF2.PdfFileReader(file(path, "rb"))
    # Iterate pages
    for i in range(0, pdf.getNumPages()):
        # Extract text from page and add to content
        content += pdf.getPage(i).extractText() + "\n"
    # Collapse whitespace
    content = " ".join(content.replace(u"\xa0", " ").strip().split())
    print(type(content))
    return content



In [0]:
print getPDFContent("/content/drive/My Drive/Citi/Citi AAdvantage Platinum Select Visa.pdf")

<type 'unicode'>
1238049189Y CARD AGREEMENTThis Card Agreement is your contract with us. It governs the use of your card and account. The enclosed Facts About Interest and Fees (ÒFact SheetÓ) is part of this Agreement. Please read this Agreement, including the Fact Sheet, care- fully. Keep them for your records. Definitionsaccount means the relationship established between you andus by this Agreement.APR means an annual percentage rate. authorized user means any person you allow to use youraccount.card means one or more cards or other access devices thatwe give you to get credit under this Agreement. This includes account numbers.we, us, and our mean Citibank, N.A., the issuer of youraccount.you, your, and yours mean the person who applied to openthe account. It also means any other person responsible for complying with this Agreement.Your Account You agree to use your account in accordance with this Agreement. You must pay us for all amounts due on your account. This Agreement is bind

In [0]:
with open('/content/drive/My Drive/Test_corpus_SUMM/06_28.xml', "r") as f:
        contents = f.read()
        soup = BeautifulSoup(contents, 'lxml')
        

required0 = soup.find_all("citances")
sentence = []
for i in required0:
    sentence.append(i.get_text())

In [0]:
sentence

["\n1 On 17 January 2006, the second respondent, Michael Saadey Abdul-Karim ('Mr Abdul-Karim'), signed an authority under s 188 of the Bankruptcy Act 1966 (Cth) ('the Act') naming and authorising the first respondent, Nicholas James Eddy ('Mr Eddy'), as his controlling trustee under Part X of the Act. An application by the applicant, the Council of the New South Wales Bar Association ('the Bar Council'), to set that authority aside was rejected by me on 3 February 2006: See Council of New South Wales Bar Association v Abdul-Karim [2006] FCA 28. \n"]

['CORPORATIONS ACT 2001 ',
 'TRADE PRACTICES ACT 1974 ',
 'INDUSTRIAL RELATIONS ACT 1988 ',
 'TRADE MARKS ACT 1995 ',
 'MUTUAL ASSISTANCE IN CRIMINAL MATTERS LEGISLATION AMENDMENT BILL 1996 ',
 'TRADE PRACTICES ACT 1974 ',
 'ACTS INTERPRETATION ACT 1901 ',
 'TRADE PRACTICES ACT 1974 ',
 'MIGRATION ACT 1958 ',
 'CORPORATIONS ACT 2001 ',
 'BANKRUPTCY ACT 1966 ',
 'FEDERAL COURT RULES 1979 ',
 'MIGRATION ACT 1958 ',
 'PATENTS ACT 1990 ',
 'TRADE PRACTICES ACT 1974 ',
 'MIGRATION ACT 1958 ',
 'NATIVE TITLE ACT 1993 ',
 'BANKRUPTCY ACT 1966 ',
 'ADMINISTRATIVE APPEALS TRIBUNAL ACT 1975 ',
 'MIGRATION ACT 1958 ',
 'TRADE PRACTICES ACT 1974 ',
 'ADMINISTRATIVE APPEALS TRIBUNAL ACT 1975 ',
 'MIGRATION ACT 1958 ',
 'MIGRATION ACT 1958 ',
 'BANKRUPTCY ACT 1966 ',
 'FEDERAL COURT RULES 1979 ',
 'TRADE PRACTICES ACT 1974 ',
 'JUDICIARY ACT 1903 ',
 'FEDERAL COURT RULES 1979 ',
 'FEDERAL COURT RULES 1979 ',
 'ADMINISTRATIVE APPEALS TRIBUNAL ACT 1975 ',
 'FEDERAL COURT RULES 1979 ',
 'FEDERAL COURT 

,170,841of,ability,absence,accident,accommodation,account,action,actor,addition,adequacy,allegation,analysis,anyone,appeal,appellant,applicant,application,area,argument,aspect,assessment,assistance,attention,authority,behalf,belief,beliefs,blanket,body,breach,building,business,campaign,canister,care,case,cause,century,certificate,...,significance,similarity,situation,sought,speculation,speeding,state,status,stress,subject,submission,subparas,subrule,success,suggestion,support,supporter,suppression,supra,surmise,system,termination,test,tort,traffic,trial,tribunal,understanding,union,unwillingness,view,violence,virtue,visa,want,weight,well,wife,witness,woman
from_06_12,1,0,0,0,1,0,0,2,0,1,0,1,0,1,0,0,1,1,1,1,1,0,0,1,1,0,1,0,1,1,1,1,0,0,1,1,1,1,0,0,...,0,1,1,1,1,1,1,0,0,1,1,1,1,0,1,1,0,1,1,1,0,0,1,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
from_06_7,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0
from_06_3,0,0,1,1,0,1,1,0,1,0,1,0,1,0,1,1,1,1,0,1,1,1,0,0,1,0,0,1,0,1,0,0,1,1,0,0,1,0,1,1,...,1,0,0,0,0,0,1,1,1,0,1,0,0,1,0,1,1,0,1,0,1,0,1,0,0,0,1,0,0,1,1,1,1,0,0,1,1,1,0,1
from_06_16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


'conduct suppression crime police duty care member community instance custody person group case negligence result damage proposition latitude discharge choice enforcement protection public harm blanket immunity officer injury subject operation number similarity policeman traffic flow accident situation canister building fire speeding driver pursuit support office course investigation injure family submission exercise contemplation rise inconsistency performance necessity action risk relief sought remedy breach fairness order decision concession discovery attention element tort form malice question intent knowledge power indifference possibility occasioning collection opinion rule respondent application process entitlement surmise speculation conjecture -170 event official distinct addition justice Order identity information applicant description purpose proceeding possession document Information contrast cause argument authority aspect extend evidence property suggestion detail body re

'conduct suppression crime police duty care member community instance custody person group case negligence result damage proposition latitude discharge choice enforcement protection public harm blanket immunity officer injury subject operation number similarity policeman traffic flow accident situation canister building fire speeding driver pursuit support office course investigation injure family submission exercise contemplation rise inconsistency performance necessity action risk relief sought remedy breach fairness order decision concession discovery attention element tort form malice question intent knowledge power indifference possibility occasioning collection opinion rule respondent application process entitlement surmise speculation conjecture -170 event official distinct addition justice Order identity information applicant description purpose proceeding possession document Information contrast cause argument authority aspect extend evidence property suggestion detail body re